In [79]:
import keras_vggface
import tensorflow as tf
import PIL.Image as Image
import numpy as np
import mtcnn
import scipy.spatial.distance as sp
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from keras_vggface import utils
from keras_vggface.vggface import VGGFace
import os
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [80]:
cd

/Users/valencialie


In [81]:
cd Desktop/CZ1016_DS2/kdrama/images/Male

/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Male


In [82]:
resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), # use vggface model that is used by resnet50
                                pooling='avg')

In [83]:
def extract_face(filename, required_size=(224, 224)):
    pixels = plt.imread(filename)
    detector = MTCNN()
    results = detector.detect_faces(pixels) # detect face inside a box
    x1, y1, width, height = results[0]['box'] 
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size) # resize to 224,224 because vgg model requires it to be that way 
    face_array = np.asarray(image)
#     plt.imshow(face_array)
#     plt.show()
    pixels = face_array.astype('float32')
    pixels = tf.expand_dims(pixels, axis=0)
    samples = utils.preprocess_input(pixels, version=2) 
    features = resnet50_features.predict(samples)
    return features

In [84]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Male"
# change the working directory to the path where the images are located
os.chdir(path)

In [85]:
actors = []

In [86]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpeg'):
          # adds only the image files ending with jpeg to the list
            actors.append(file.name)

In [87]:
len(actors)

576

In [88]:
data = {}

for actor in actors:
    try:
        feat = extract_face(actor)
        data[actor] = feat
    except:
        data[actor] = "Blank"

In [89]:
len(data)

576

In [90]:
keys = list(data.keys())

In [91]:
values = list(data.values())

In [92]:
d = { label: value for label, value in zip(keys, values) }

In [93]:
df = pd.DataFrame(d.items(), columns=['Actor', 'Array'])

In [94]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top10 = []
overall = []

In [95]:
len(d)

576

In [96]:
for j in range(0,576):
    for i in range(0,576):
        if i == j:
            continue
        else:
            dist = sp.cosine(values[i], values[j])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top10 = dist_list[:10]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top10.extend(sorted_list1[:10])
    overall.append(top10)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top10 = []

In [97]:
df1 = pd.DataFrame(overall,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '6thLeast_Array_Diff', '7thLeast_Array_Diff','8thLeast_Array_Diff', '9thLeast_Array_Diff', '10thLeast_Array_Diff','1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch', '6thMatch', '7thMatch', '8thMatch', '9thMatch', '10thMatch'])

In [98]:
df2 = pd.concat([df, df1], axis = 1)

In [99]:
df2

,Actor,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,6thLeast_Array_Diff,7thLeast_Array_Diff,8thLeast_Array_Diff,...,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch,6thMatch,7thMatch,8thMatch,9thMatch,10thMatch
0,Lee_Kyu_Han.jpeg,"[[0.0, 0.06436636, 0.0, 0.0, 2.1942368, 0.1459...",0.296106,0.317963,0.331765,0.346518,0.346633,0.353566,0.358768,0.361260,...,390,171,418,382,281,16,266,568,488,2
1,Im_Shi_Wan.jpeg,"[[0.0, 0.0, 1.957572, 0.0, 1.343608, 2.250862,...",0.345610,0.361965,0.364178,0.371152,0.376647,0.382276,0.383490,0.385891,...,545,5,402,451,378,431,330,292,548,168
2,Jae_Hee.jpeg,"[[0.0, 1.1606871, 0.0, 0.8187124, 0.12278407, ...",0.294929,0.308964,0.319130,0.348895,0.359478,0.359915,0.361896,0.362959,...,240,568,390,342,545,493,170,315,281,0
3,Kim_Sung_Oh.jpeg,"[[0.0744203, 0.27144027, 3.1494617, 0.13220347...",0.349409,0.376483,0.399360,0.411211,0.412942,0.426894,0.435751,0.441681,...,568,390,321,66,108,315,334,418,57,125
4,Jo_Hee_Bong.jpeg,"[[2.405822, 0.25564858, 0.0, 0.62448454, 0.878...",0.361091,0.392543,0.396306,0.419863,0.427464,0.432157,0.437181,0.438818,...,433,48,139,123,174,437,454,190,124,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,Dong_Ha.jpeg,"[[0.0, 2.4758608, 2.1770866, 0.45120138, 0.877...",0.308361,0.324596,0.334146,0.353958,0.361262,0.362418,0.365035,0.368109,...,291,545,33,198,307,126,425,384,496,235
572,Sung_Ji_Ru.jpeg,"[[0.013988101, 1.4476267, 1.390254, 0.25296915...",0.342572,0.355290,0.400561,0.402929,0.407040,0.411747,0.415580,0.415895,...,73,172,437,59,445,409,41,71,360,264
573,Kim_Hyung_Min.jpeg,"[[0.3368032, 4.141306, 2.5392754, 3.21328, 1.1...",0.356046,0.358556,0.359138,0.363955,0.372793,0.380507,0.395548,0.398931,...,540,499,180,530,171,545,198,381,99,490
574,Park_Byung_Eun.jpeg,"[[0.10699663, 0.0, 0.0, 0.819224, 1.1563241, 7...",0.346609,0.357618,0.363873,0.367063,0.367087,0.368965,0.371071,0.374148,...,15,362,401,168,372,492,345,91,447,366


In [100]:
bestmatch = []

for i in range(0,576):
    bestmatch.append(df2.iloc[df2['1stMatch'][i], 0])
    
secondmatch = []

for i in range(0,576):
    secondmatch.append(df2.iloc[df2['2ndMatch'][i], 0])
    
thirdmatch = []

for i in range(0,576):
    thirdmatch.append(df2.iloc[df2['3rdMatch'][i], 0])
    
fourthmatch = []

for i in range(0,576):
    fourthmatch.append(df2.iloc[df2['4thMatch'][i], 0])

fifthmatch = []

for i in range(0,576):
    fifthmatch.append(df2.iloc[df2['5thMatch'][i], 0])
    
sixthmatch = []

for i in range(0,576):
    sixthmatch.append(df2.iloc[df2['6thMatch'][i], 0])
    
seventhmatch = []

for i in range(0,576):
    seventhmatch.append(df2.iloc[df2['7thMatch'][i], 0])
    
eighthmatch = []

for i in range(0,576):
    eighthmatch.append(df2.iloc[df2['8thMatch'][i], 0])
    
ninthmatch = []

for i in range(0,576):
    ninthmatch.append(df2.iloc[df2['9thMatch'][i], 0])
    
tenthmatch = []

for i in range(0,576):
    tenthmatch.append(df2.iloc[df2['10thMatch'][i], 0])

df3 = pd.DataFrame(bestmatch,columns=['Best_Match_Actor'])
df4 = pd.DataFrame(secondmatch,columns=['Second_Match_Actor'])
df5 = pd.DataFrame(thirdmatch,columns=['Third_Match_Actor'])
df6 = pd.DataFrame(fourthmatch,columns=['Fourth_Match_Actor'])
df7 = pd.DataFrame(fifthmatch,columns=['Fifth_Match_Actor'])
df8 = pd.DataFrame(sixthmatch,columns=['Sixth_Match_Actor'])
df9 = pd.DataFrame(seventhmatch,columns=['Seventh_Match_Actor'])
df10 = pd.DataFrame(eighthmatch,columns=['Eighth_Match_Actor'])
df11 = pd.DataFrame(ninthmatch,columns=['Ninth_Match_Actor'])
df12 = pd.DataFrame(tenthmatch,columns=['Tenth_Match_Actor'])

df13 = pd.concat([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12], axis = 1)

In [101]:
cd

/Users/valencialie


In [102]:
cd Desktop/CZ1016_DS2/kdrama/images/Female

/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Female


In [103]:
actresses = []

In [104]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Female"
# change the working directory to the path where the images are located
os.chdir(path)

In [105]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpeg'):
          # adds only the image files ending with jpeg to the list
            actresses.append(file.name)

In [106]:
len(actresses)

474

In [107]:
dataF = {}

for actress in actresses:
    try:
        feat = extract_face(actress)
        dataF[actress] = feat
    except:
        dataF[actress] = "Blank"

In [108]:
label = list(dataF.keys())
values = list(dataF.values())

In [109]:
f = { label: value for label, value in zip(label, values) }

In [110]:
dF = pd.DataFrame(f.items(), columns=['Actress', 'Array'])

In [111]:
len(dF)

474

In [112]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top10 = []
overall = []

In [113]:
for j in range(0,474):
    for i in range(0,474):
        if i == j:
            continue
        else:
            dist = sp.cosine(values[i], values[j])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top10 = dist_list[:10]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top10.extend(sorted_list1[:10])
    overall.append(top10)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top10 = []

In [114]:
overall

[[0.2759072780609131,
  0.30143624544143677,
  0.30158185958862305,
  0.3132932186126709,
  0.3174683451652527,
  0.32123154401779175,
  0.32126545906066895,
  0.3213655948638916,
  0.3339293599128723,
  0.33499354124069214,
  349,
  377,
  117,
  76,
  468,
  288,
  304,
  58,
  438,
  102],
 [0.3052330017089844,
  0.34038472175598145,
  0.3465281128883362,
  0.3602760434150696,
  0.3770729899406433,
  0.3776063919067383,
  0.3805725574493408,
  0.3853731155395508,
  0.38545799255371094,
  0.3862453103065491,
  148,
  268,
  101,
  91,
  24,
  86,
  84,
  212,
  417,
  388],
 [0.3711179494857788,
  0.39912086725234985,
  0.40277528762817383,
  0.41484469175338745,
  0.4197509288787842,
  0.42957931756973267,
  0.43236273527145386,
  0.43443483114242554,
  0.44191235303878784,
  0.4495909810066223,
  67,
  263,
  49,
  162,
  321,
  442,
  62,
  228,
  290,
  260],
 [0.30561214685440063,
  0.3172149658203125,
  0.31926655769348145,
  0.34041398763656616,
  0.3438798189163208,
  0.34837

In [115]:
dF1 = pd.DataFrame(overall,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '6thLeast_Array_Diff', '7thLeast_Array_Diff','8thLeast_Array_Diff', '9thLeast_Array_Diff', '10thLeast_Array_Diff','1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch', '6thMatch', '7thMatch', '8thMatch', '9thMatch', '10thMatch'])

In [116]:
dF2 = pd.concat([dF, dF1], axis = 1)

In [117]:
dF2

,Actress,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,6thLeast_Array_Diff,7thLeast_Array_Diff,8thLeast_Array_Diff,...,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch,6thMatch,7thMatch,8thMatch,9thMatch,10thMatch
0,Jung_Yoo_Jin.jpeg,"[[0.0, 0.2543437, 0.0, 4.4913435, 3.635107, 0....",0.275907,0.301436,0.301582,0.313293,0.317468,0.321232,0.321265,0.321366,...,349,377,117,76,468,288,304,58,438,102
1,Seo_Yi_Ahn.jpeg,"[[1.1509761, 0.047182605, 2.0433936, 0.5478225...",0.305233,0.340385,0.346528,0.360276,0.377073,0.377606,0.380573,0.385373,...,148,268,101,91,24,86,84,212,417,388
2,Kang_Ye_Na.jpeg,"[[1.9589509, 0.0, 0.40266016, 0.42564636, 2.02...",0.371118,0.399121,0.402775,0.414845,0.419751,0.429579,0.432363,0.434435,...,67,263,49,162,321,442,62,228,290,260
3,Kim_Hyun_Soo.jpeg,"[[0.0, 0.14416984, 4.598352, 0.061391816, 5.13...",0.305612,0.317215,0.319267,0.340414,0.343880,0.348375,0.352641,0.355292,...,353,329,47,383,135,406,183,207,366,79
4,Yoo_Hye_Ri.jpeg,"[[0.47068676, 0.82373685, 1.8157511, 2.4260333...",0.401623,0.409698,0.411395,0.416729,0.426655,0.426857,0.432220,0.441863,...,355,433,354,159,344,356,221,97,438,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Hwang_Seung_Eon.jpeg,"[[0.0, 0.012250069, 2.016425, 0.021052122, 2.2...",0.226859,0.237822,0.238910,0.275411,0.289282,0.302689,0.305662,0.307152,...,183,15,180,79,93,236,56,361,383,329
470,Chae_Soo_Bin.jpeg,"[[0.0, 0.27327007, 0.0, 0.0, 3.1540382, 0.0, 0...",0.314409,0.314584,0.316973,0.320586,0.345937,0.351958,0.352943,0.353739,...,430,394,366,465,16,422,18,37,333,179
471,Yang__Jin_Sung.jpeg,"[[1.9149007, 0.37245303, 0.0, 1.9541427, 4.512...",0.323316,0.332633,0.360458,0.378607,0.384275,0.384618,0.387496,0.393011,...,292,378,73,352,462,13,25,24,428,101
472,Park_Bo_Young.jpeg,"[[0.0, 0.38084894, 0.0, 0.0, 6.845794, 0.0, 0....",0.246092,0.267348,0.271500,0.276044,0.276334,0.282800,0.297798,0.298549,...,115,373,58,343,394,345,259,139,352,382


In [118]:
bestmatch = []

for i in range(0,474):
    bestmatch.append(dF2.iloc[dF2['1stMatch'][i], 0])
    
secondmatch = []

for i in range(0,474):
    secondmatch.append(dF2.iloc[dF2['2ndMatch'][i], 0])
    
thirdmatch = []

for i in range(0,474):
    thirdmatch.append(dF2.iloc[dF2['3rdMatch'][i], 0])
    
fourthmatch = []

for i in range(0,474):
    fourthmatch.append(dF2.iloc[dF2['4thMatch'][i], 0])

fifthmatch = []

for i in range(0,474):
    fifthmatch.append(dF2.iloc[dF2['5thMatch'][i], 0])
    
sixthmatch = []

for i in range(0,474):
    sixthmatch.append(dF2.iloc[dF2['6thMatch'][i], 0])
    
seventhmatch = []

for i in range(0,474):
    seventhmatch.append(dF2.iloc[dF2['7thMatch'][i], 0])
    
eighthmatch = []

for i in range(0,474):
    eighthmatch.append(dF2.iloc[dF2['8thMatch'][i], 0])
    
ninthmatch = []

for i in range(0,474):
    ninthmatch.append(dF2.iloc[dF2['9thMatch'][i], 0])
    
tenthmatch = []

for i in range(0,474):
    tenthmatch.append(dF2.iloc[dF2['10thMatch'][i], 0])

dF3 = pd.DataFrame(bestmatch,columns=['Best_Match_Actor'])
dF4 = pd.DataFrame(secondmatch,columns=['Second_Match_Actor'])
dF5 = pd.DataFrame(thirdmatch,columns=['Third_Match_Actor'])
dF6 = pd.DataFrame(fourthmatch,columns=['Fourth_Match_Actor'])
dF7 = pd.DataFrame(fifthmatch,columns=['Fifth_Match_Actor'])
dF8 = pd.DataFrame(sixthmatch,columns=['Sixth_Match_Actor'])
dF9 = pd.DataFrame(seventhmatch,columns=['Seventh_Match_Actor'])
dF10 = pd.DataFrame(eighthmatch,columns=['Eighth_Match_Actor'])
dF11 = pd.DataFrame(ninthmatch,columns=['Ninth_Match_Actor'])
dF12 = pd.DataFrame(tenthmatch,columns=['Tenth_Match_Actor'])

dF13 = pd.concat([dF2, dF3, dF4, dF5, dF6, dF7, dF8, dF9, dF10, dF11, dF12], axis = 1)

In [78]:
dF13

,Actress,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,6thLeast_Array_Diff,7thLeast_Array_Diff,8thLeast_Array_Diff,...,Best_Match_Actor,Second_Match_Actor,Third_Match_Actor,Fourth_Match_Actor,Fifth_Match_Actor,Sixth_Match_Actor,Seventh_Match_Actor,Eighth_Match_Actor,Ninth_Match_Actor,Tenth_Match_Actor
0,Jung_Yoo_Jin.jpeg,"[[0.0, 0.2543437, 0.0, 4.4913435, 3.635107, 0....",0.275907,0.301436,0.301582,0.313293,0.317468,0.321232,0.321265,0.321366,...,Kwon_Na_Ra.jpeg,Song_Hye_Kyo.jpeg,Yoo_Da_In.jpeg,Son_Ye_Jin.jpeg,Shin_Se_Kyung.jpeg,Lee__Ha_Na_.jpeg,Kim_Ji_Won.jpeg,Kim_Yoon_Hye.jpeg,Han_Bo_Reum.jpeg,Cho_Yeo_Jung.jpeg
1,Seo_Yi_Ahn.jpeg,"[[1.1509761, 0.047182605, 2.0433936, 0.5478225...",0.305233,0.340385,0.346528,0.360276,0.377073,0.377606,0.380573,0.385373,...,Son_Hwa_Ryung.jpeg,Kim_Hyun_Joo.jpeg,Lee_Bo_Young.jpeg,Lee_Hwi_Hyang.jpeg,Go_Yoon_Jung.jpeg,Gil_Eun_Hye.jpeg,Kim_Tae_Hee.jpeg,Seo_Ji_Hye.jpeg,Jung_Ryeo_Won.jpeg,Han__Sun_Hwa.jpeg
2,Kang_Ye_Na.jpeg,"[[1.9589509, 0.0, 0.40266016, 0.42564636, 2.02...",0.371118,0.399121,0.402775,0.414845,0.419751,0.429579,0.432363,0.434435,...,Go_Ah_Ra.jpeg,Uee.jpeg,Sa_Hyun_Jin.jpeg,Lee_Shi_Young.jpeg,Im_Sung_Eon.jpeg,Han_Hyo_Joo.jpeg,Oh_Na_Ra.jpeg,Lee_Hee_Won.jpeg,Jang_Young_Nam.jpeg,Seo_Woo.jpeg
3,Kim_Hyun_Soo.jpeg,"[[0.0, 0.14416984, 4.598352, 0.061391816, 5.13...",0.305612,0.317215,0.319267,0.340414,0.343880,0.348375,0.352641,0.355292,...,Kim_So_Hyun.jpeg,Jung_Da_Bin.jpeg,Ryu_Hwa_Young.jpeg,Yoon_Eun_Hye.jpeg,Na_Eun_Saem.jpeg,Go_Ah_Sung.jpeg,Son_Dam_Bi.jpeg,Han_Ji_Eun.jpeg,Park_Joo_Mi.jpeg,Han_Groo.jpeg
4,Yoo_Hye_Ri.jpeg,"[[0.47068676, 0.82373685, 1.8157511, 2.4260333...",0.401623,0.409698,0.411395,0.416729,0.426655,0.426857,0.432220,0.441863,...,Shim_Hye_Jin.jpeg,Kim_Sun_Ah.jpeg,Yoo_Seo_Jin.jpeg,Kim_Min_Seo.jpeg,Choi_Soo_Rin.jpeg,Kim_Jee_Soo.jpeg,Chae_Jung_An.jpeg,Jin_Ye_Ju.jpeg,Han_Bo_Reum.jpeg,Kim_Nam_Joo.jpeg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Hwang_Seung_Eon.jpeg,"[[0.0, 0.012250069, 2.016425, 0.021052122, 2.2...",0.226859,0.237822,0.238910,0.275411,0.289282,0.302689,0.305662,0.307152,...,Son_Dam_Bi.jpeg,Im_Ji_Yeon.jpeg,Park_Joo_Hyun.jpeg,Han_Groo.jpeg,Son_Sung_Yoon.jpeg,Shin_Dong_Mi.jpeg,Hong_Ji_Yoon.jpeg,Kim_Hyang_Gi_.jpeg,Yoon_Eun_Hye.jpeg,Jung_Da_Bin.jpeg
470,Chae_Soo_Bin.jpeg,"[[0.0, 0.27327007, 0.0, 0.0, 3.1540382, 0.0, 0...",0.314409,0.314584,0.316973,0.320586,0.345937,0.351958,0.352943,0.353739,...,Jeon_Yeo_Bin.jpeg,Jin_Se_Yeon.jpeg,Park_Joo_Mi.jpeg,Kim_Hye_Joon.jpeg,Shin_Ye_Eun.jpeg,So_Ju_Yeon.jpeg,Lee_Seung_Yeon.jpeg,Jo_Hye_Joo.jpeg,Esom.jpeg,Nam_Kyung_Min.jpeg
471,Yang__Jin_Sung.jpeg,"[[1.9149007, 0.37245303, 0.0, 1.9541427, 4.512...",0.323316,0.332633,0.360458,0.378607,0.384275,0.384618,0.387496,0.393011,...,Kim_Yoo_Jung.jpeg,Jung__Soo_Young.jpeg,Nam_Ji_Hyun.jpeg,Jung_Da_Sol.jpeg,Moon_So_Ri.jpeg,Hwang_Jung_Eum.jpeg,Lee_Young_Eun.jpeg,Go_Yoon_Jung.jpeg,Lee_Yeol_Eum.jpeg,Lee_Bo_Young.jpeg
472,Park_Bo_Young.jpeg,"[[0.0, 0.38084894, 0.0, 0.0, 6.845794, 0.0, 0....",0.246092,0.267348,0.271500,0.276044,0.276334,0.282800,0.297798,0.298549,...,Park_Ji_Hyun.jpeg,Oh_Yeon_Seo.jpeg,Kim_Yoon_Hye.jpeg,Kim_Yoo_Ri.jpeg,Jin_Se_Yeon.jpeg,Baek_Soo_Min.jpeg,Jung_Yu_Mi.jpeg,Lee_Joo_Young.jpeg,Jung_Da_Sol.jpeg,Jang_Hee_Jin.jpeg
